In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import gzip
from multiprocessing import Pool
import os
import pandas as pd 
import re


In [2]:
import os

In [3]:
vcf_file = 'arp-v2.1-mc-chm13.vcf'


In [4]:
import pickle

In [5]:
def extract_info(info_str, key):
    for field in info_str.split(';'):
        if field.startswith(key + '='):
            return field[len(key)+1:]
    return None

In [6]:
def allele_lengths(ref, alt):
    alts = alt.split(',')
    return [len(a) for a in alts]

In [7]:
all_sites = []
with open(vcf_file, 'r') as vcf:
    for line in vcf:
        if not line.startswith('#'):
            columns = line.strip().split('\t')
            ref = columns[3]
            alt = columns[4]
            lengths = allele_lengths(ref, alt)
            
            if any((l-len(ref)) > 50 for l in lengths):
                all_sites.append(line.strip().split('\t'))

KeyboardInterrupt: 

In [8]:
all_sites_df = pd.DataFrame(all_sites)
all_sites_df.shape

(49648, 53)

In [9]:
#unique svs:
unique_svs_path = '/mnt/h700/omixlab/workflows/bipin_works/VCF_ANALYSIS_APR_HPRC_CPR_SPLIT/COMPARE_SVS/TRUVARI_ANALYSIS/HRPC-CPC_APR_COMPARISON/INS-DEL/output_dir_0.8/fp.vcf'

lines = 0
unique_variants = set()
with open(unique_svs_path, 'r') as f:
    for line in f:
        if not line.startswith('#'):
            items = line.split()
            unique_variants.add((items[0].split('.')[-1], items[1]))
            lines +=1
            

In [10]:
unique_df = pd.DataFrame(unique_variants, columns = ['chrom', 'pos' ])

In [8]:
interesting_alleles = []
with open(vcf_file, 'r') as vcf:
    for line in vcf:
        if not line.startswith('#'):  # Exclude header lines
            columns = line.strip().split('\t')
            ref = columns[3]
            alt = columns[4]
            
            lengths = allele_lengths(ref, alt)

            if ((len(lengths) >= 5 and any((l-len(ref)) > 10000 for l in lengths)) ):
                interesting_alleles.append(line.strip())


In [9]:
len(interesting_alleles)

715

In [10]:
with open('complex_sites.csv', 'w') as f:
    for allele in interesting_alleles:
        f.write(f'{allele.split()[0]} {allele.split()[1]}\n')
        
    

In [10]:
with open('interesting_sites.tsv' , 'w') as f:
    f.write('chr\tpos\talt\n')
    for site in interesting_alleles:
        items = site.split('\t')
        sv_lengths = [str(len(item)) for item in items[4].split(',')]
        f.write(f'{items[0]}\t{items[1]}\t{",".join((sv_lengths))}\n')

In [11]:
import pandas as pd

In [12]:
def get_df(sites):
    data = []
    for site in sites:
        data.append((site.split()[0], int(site.split()[1])))
     
    return pd.DataFrame(data, columns=['chrom', 'pos'])

In [13]:
all_sites_df[1] = pd.to_numeric(all_sites_df[1])

In [14]:
site_df = get_df(interesting_alleles)

In [38]:
site_df['chrom'] = site_df['chrom'].astype(str)
unique_df['chrom'] = unique_df['chrom'].astype(str)
unique_df['pos'] = unique_df['pos'].astype(int)

chrom    object
pos      object
dtype: object

In [40]:
unique_apr_sites = unique_df.merge(site_df, on=['chrom', 'pos'], how='inner')

In [14]:
window_size = 50000
regions = []
for i,site in site_df.iterrows():

    chrom = site['chrom']
    pos = site['pos']

    sites_in_window = all_sites_df.loc[(all_sites_df[0] == chrom ) & (all_sites_df[1]>pos-window_size) & (all_sites_df[1]<pos+window_size)]
    
    if sites_in_window.shape[0] >= 2:
        regions.append([chrom, pos-window_size, pos+window_size])
        
       
    
    
        

In [44]:
window_size = 50000
unique_regions = []
for i,site in unique_apr_sites.iterrows():
    print(i, end='\r')
    chrom = site['chrom']
    pos = site['pos']

    sites_in_window = all_sites_df.loc[(all_sites_df[0] == chrom ) & (all_sites_df[1]>pos-window_size) & (all_sites_df[1]<pos+window_size)]
    
    if sites_in_window.shape[0] >= 2:
        unique_regions.append([chrom, pos-window_size, pos+window_size])

In [45]:
len(unique_regions)

510

In [46]:
def extract_genes_from_gff3(filename):
    genes = []
    with open(filename, 'r') as gff:
        for line in gff:
            if not line.startswith("#"):  # Ignore comment lines
                parts = line.strip().split("\t")
                if parts[2] == "gene" or parts[2] == "pseudogene":
                    chrom = parts[0]
                    start = int(parts[3])
                    end = int(parts[4])
                    attributes = parts[8].split(";")
                    gene_name = None
                    is_protein_coding = False
                    for attribute in attributes:
                        if attribute.startswith("ID="):
                            gene_name = attribute.split("=")[1]
                        if attribute == "gene_biotype=protein_coding":
                                is_protein_coding = True
                            
                    if gene_name:
                        genes.append((chrom,start, end, '-'.join(gene_name.split('-')[1:]), is_protein_coding))
    return genes


In [47]:
def variants_within_flank(variants, genes, flank_size):
    result = {}
    
    for i,line in variants.iterrows():
        chrom = line['CHROM']
        pos = int(line['POS'])
        start_flank = pos - flank_size
        end_flank = pos + flank_size
        
        # Check if genes are within the flanking region
        for start, end, gene_name in genes.get(chrom, []):
                    if start < end_flank and end > start_flank:
                        key = (chrom, pos)
                        if key not in result:
                            result[key] = []
                        result[key].append(gene_name)
    return result

In [48]:
def count_genes(gff3_file):
    with open(gff3_file, 'r') as f:
        # Filter out comment lines and split by tabs
        lines = [line.split('\t') for line in f if not line.startswith('#')]
        
        # Count lines where the third column is "gene"
        return sum(1 for line in lines if len(line) > 2 and line[2] == 'gene')

In [49]:
def extract_genes_from_region(filename, chrom_filter, start_filter, end_filter):
    all_genes = []
    pc_genes = []
    with open(filename, 'r') as gff:
        for line in gff:
            if not line.startswith("#"):  # Ignore comment lines
                parts = line.strip().split("\t")
                chrom = parts[0]
                start = int(parts[3])
                end = int(parts[4])
                
                # Filter for the specific region
                if chrom == chrom_filter and (parts[2] == "gene" or parts[2] == "pseudogene") and start_filter <= start and end_filter >= end:
                    attributes = parts[8].split(";")
                    gene_name = None
                    is_protein_coding = False
                    for attribute in attributes:
                        if attribute.startswith("ID="):
                            gene_name = attribute.split("=")[1]
                        if attribute == "gene_biotype=protein_coding":
                                is_protein_coding = True
                                
                        
                    if gene_name:
                        all_genes.append(gene_name)
                        
                    if gene_name and is_protein_coding:
                        pc_genes.append(gene_name)
    return all_genes, pc_genes


In [51]:
def extract_genes_from_region(genes, chrom_filter, start_filter, end_filter):
    all_genes = []
    pc_genes = []
   
    for line in genes:
        if not line.startswith("#"):  # Ignore comment lines
            parts = line.strip().split("\t")
            chrom = parts[0]
            start = int(parts[3])
            end = int(parts[4])
            
            # Filter for the specific region
            if chrom == chrom_filter and (parts[2] == "gene" or parts[2] == "pseudogene") and start_filter <= start and end_filter >= end:
                attributes = parts[8].split(";")
                gene_name = None
                is_protein_coding = False
                for attribute in attributes:
                    if attribute.startswith("ID="):
                        gene_name = attribute.split("=")[1]
                    if attribute == "gene_biotype=protein_coding":
                            is_protein_coding = True
                            
                    
                if gene_name:
                    all_genes.append(gene_name)
                    
                if gene_name and is_protein_coding:
                    pc_genes.append(gene_name)
    return all_genes, pc_genes

In [52]:

gff3_file = "chm13_genomic_chr.gff3"

all_genes_df = pd.DataFrame(extract_genes_from_gff3(gff3_file))
# total_genes = count_genes(gff3_file)

In [50]:
recessive_genes = pd.read_excel('Recessive genes_GA gene list.xlsx')

In [51]:
recessive_gene_set = set(recessive_genes['Gene Name'])

In [54]:
#unique analysis:
all_pc_genes = []
for region in unique_regions:
    chr = region[0]
    start = region[1]
    end = region[2]

    temp_genes_df = all_genes_df.loc[(all_genes_df[0] == chr) & (pd.to_numeric(all_genes_df[1]) > start) & (pd.to_numeric(all_genes_df[2]) < end)   ]
    all = list(temp_genes_df[3])
    pc = list(temp_genes_df.loc[temp_genes_df[4], 3])

    region.append(','.join(all))
    region.append(len(all))
    region.append(','.join(pc))
    region.append(len(pc))
    
    
    all_pc_genes.extend(pc)

In [29]:
all_pc_genes = []
for region in regions:
    chr = region[0]
    start = region[1]
    end = region[2]

    temp_genes_df = all_genes_df.loc[(all_genes_df[0] == chr) & (pd.to_numeric(all_genes_df[1]) > start) & (pd.to_numeric(all_genes_df[2]) < end)   ]
    all = list(temp_genes_df[3])
    pc = list(temp_genes_df.loc[temp_genes_df[4], 3])

    region.append(','.join(all))
    region.append(len(all))
    region.append(','.join(pc))
    region.append(len(pc))
    
    
    all_pc_genes.extend(pc)


In [58]:
pd.DataFrame(unique_regions, columns=['chr','start','end', 'all_genes','no of genes', 'protein_genes', 'no of pc genes']).to_csv('unique_apr_genes_overlap.csv', index=None)

In [56]:
with open('unique_apr_region_genes.csv', 'w') as f:
    for gene in all_pc_genes:
        f.write(f'{gene}\n')

In [67]:
df_complex_regions = pd.DataFrame(regions, columns=['chr','start','end', 'all_genes','no of genes', 'protein_genes', 'no of pc genes', 'recessive_genes', 'no of recessive genes'])

In [ ]:
syndromic_regions = pd.read_excel('DECIPHER syndromes.xlsx')
syndromic_regions['Chr'] =  syndromic_regions['Chr'].apply(lambda x : f"chr{x}")
syndromic_regions = syndromic_regions[['Chr', 'Start', 'End']]

In [79]:
syndromic_all_genes = []
pc_all_genes = []
for i,region in syndromic_regions.iterrows():
    chr = region['Chr']
    start = region['Start']
    end = region['End']

    temp_genes_df = all_genes_df.loc[(all_genes_df[0] == chr) & (pd.to_numeric(all_genes_df[1]) > start) & (pd.to_numeric(all_genes_df[2]) < end)   ]
    all = list(temp_genes_df[3])
    pc = list(temp_genes_df.loc[temp_genes_df[4], 3])
    
    syndromic_all_genes.extend(all)
    pc_all_genes.extend(pc)

    

In [88]:
for region in regions:
    chr = region[0]
    start = region[1]
    end = region[2]

    temp_genes_df = all_genes_df.loc[(all_genes_df[0] == chr) & (pd.to_numeric(all_genes_df[1]) > start) & (pd.to_numeric(all_genes_df[2]) < end)   ]
    all = list(temp_genes_df[3])
    pc = list(temp_genes_df.loc[temp_genes_df[4], 3])

    region.append(','.join(all))
    region.append(len(all))
    region.append(','.join(pc))
    region.append(len(pc))
    region.append(','.join(set(pc).intersection(set(pc_all_genes))))
    region.append(len(set(pc).intersection(set(pc_all_genes))))

In [90]:
pd.DataFrame(regions, columns=['chr','start','end', 'all_genes','no of genes', 'protein_genes', 'no of pc genes', 'syndromic genes', 'no of syndromic genes']).to_csv('syndromic_genes_overlap.csv', index=None)

In [32]:
syndromic_regions_with_sites = []
for i, row in syndromic_regions.iterrows():
    sites_within_region = all_sites_df.loc[(all_sites_df[0] == row['Chr']) & (all_sites_df[1] > row['Start']) & (all_sites_df[1] < row['End'])]
    
    if sites_within_region.shape[0] > 0:
        syndromic_regions_with_sites.append(row)

In [37]:
syndromic_regions_with_sites_df=pd.DataFrame(syndromic_regions_with_sites)

In [72]:
df_complex_regions

,chr,start,end,all_genes,no of genes,protein_genes,no of pc genes,recessive_genes,no of recessive genes
0,chr1,2046666,2146666,MMEL1-AS1,1,,0,,0
1,chr1,2052619,2152619,MMEL1-AS1,1,,0,,0
2,chr1,2052916,2152916,MMEL1-AS1,1,,0,,0
3,chr1,5482193,5582193,,0,,0,,0
4,chr1,6997051,7097051,LOC105376691,1,,0,,0
...,...,...,...,...,...,...,...,...,...
641,chrY,28732195,28832195,,0,,0,,0
642,chrY,28756400,28856400,,0,,0,,0
643,chrY,29285944,29385944,"LOC124908564,LOC124908677",2,,0,,0
644,chrY,29337838,29437838,LOC124908677,1,,0,,0


In [73]:
syndromic_genes = []
for region in syndromic_regions_with_sites:
    one_region = []
    chr = region['Chr']
    start = region["Start"]
    end = region['End']
    temp_genes_df = df_complex_regions.loc[(df_complex_regions['chr'] == chr) & (pd.to_numeric(df_complex_regions['start']) > start) & (pd.to_numeric(df_complex_regions['end']) < end)]
    
    for i, row in temp_genes_df.iterrows():
        syndromic_genes.append(row)
    
    
    

In [76]:
pd.DataFrame(syndromic_genes).to_csv('regions_within_syndromic.csv', index=None)

In [57]:
syndromic_genes = []
for region in syndromic_regions_with_sites:
    one_region = []
    chr = region['Chr']
    start = region["Start"]
    end = region['End']
    temp_genes_df = all_genes_df.loc[(all_genes_df[0] == chr) & (pd.to_numeric(all_genes_df[1]) > start) & (pd.to_numeric(all_genes_df[2]) < end)]
    all = list(temp_genes_df[3])
    pc = list(temp_genes_df.loc[temp_genes_df[4], 3])

    one_region.extend([chr, start, end])
    
    one_region.append(','.join(all))
    one_region.append(len(all))
    one_region.append(','.join(pc))
    one_region.append(len(pc))
    one_region.append(''.join(set(pc).intersection(recessive_gene_set)))
    one_region.append(len(set(pc).intersection(recessive_gene_set)))
    
    syndromic_genes.append(one_region)
    

In [61]:
pd.DataFrame(syndromic_genes, columns=['chr','start','end', 'all_genes','no of genes', 'protein_genes', 'no of pc genes', 'recessive_genes', 'no of recessive genes']).to_csv('syndromic_genes.csv', index=None)

In [104]:
range = 500000

In [105]:
result = variants_within_flank(df, genes, range)

In [4]:
rows = [(key, val) for key, values in result.items() for val in values]

# Create a DataFrame
gene_df = pd.DataFrame(rows, columns=['loc', 'gene'])


recessive_genes = pd.read_excel('Recessive genes_GA gene list.xlsx')
intersect = gene_df.merge(recessive_genes, how='left', left_on='gene', right_on='Gene Name').dropna().drop(['Gene Name'], axis=1)
intersect_dict = {}
for i, row in intersect.iterrows():
    if row['loc'] not in intersect_dict:
        intersect_dict[row['loc']] = [] 
    intersect_dict[row['loc']].append(str(row['gene']))


NameError: name 'result' is not defined

In [107]:
# intersect.to_csv('gene_recessive_intersection.csv', index= None)
# intersect.groupby('loc').count().sort_values('gene', ascending=False).to_csv('count_recessive.csv', index=None)

In [108]:
from scipy.stats import fisher_exact
def pvalue_and_odds_ratio(total, first_set, second_set, overlap):
    table = [[overlap, first_set-overlap], [second_set-overlap, total-first_set-second_set+overlap]]
    odds_ratio, p_value = fisher_exact(table)
    return odds_ratio, p_value

In [109]:
pvalues= []
for k,v in result.items():
    site_genes = len(set(v))
    if k in intersect_dict:
        intersected_genes = len(set(intersect_dict[k]))
    else:
        intersected_genes = 0
    total_recessive = recessive_genes.shape[0]
    
    o, p = pvalue_and_odds_ratio(total_genes, site_genes, total_recessive, intersected_genes)
    
    
    if k in intersect_dict:
        pvalues.append((k, p, o,site_genes,intersected_genes,','.join(set(v)), ','.join(set(intersect_dict[k]))))
    else:
        pvalues.append((k, p, o,site_genes,intersected_genes,','.join(set(v)), ''))
    
    

In [110]:
p_df = pd.DataFrame(pvalues, columns = ['loc', 'pvalue', 'odds_ratio','number of genes in site', 'number of recessive genes in site', 'all genes', 'genes'])

In [111]:
p_df.to_csv(f'recessive_overlap_{range*2}.csv')

In [112]:
(p_df.loc[p_df['loc']==('chr13', 12990689)]['genes'].str.len())

50    0
Name: genes, dtype: int64

In [113]:
p_df.to_csv('pvalue_per_site.csv', index=None)

In [114]:
intersect['gene'].unique().shape[0]

138

7018

In [ ]:
from scipy.stats import fisher_exact
def pvalue_and_odds_ratio(total, first_set, second_set, overlap):
    table = [[overlap, first_set-overlap], [second_set-overlap, total-first_set-second_set+overlap]]
    odds_ratio, p_value = fisher_exact(table)

In [5]:
duplication_ratio_matrix = pd.read_csv('dup_ratio.csv')
intersect=(set(duplication_ratio_matrix['Gene Name']).intersection(set(recessive_genes['Gene Name'])))

In [6]:
duplication_ratio_matrix['sum'] = duplication_ratio_matrix[duplication_ratio_matrix['Gene Name'].isin(intersect)].iloc[:, 1:].sum(axis=1)
duplication_ratio_matrix=duplication_ratio_matrix.sort_values('sum', ascending=False)

In [7]:
duplication_ratio_matrix.iloc[:10]

,Gene Name,brother-brother/1,brother-brother/2,father-father/1,father-father/2,gp001-gp001/1,gp001-gp001/2,gp002-gp002/1,gp002-gp002/2,gp003-gp003/1,...,gp038-gp038/2,gp039-gp039/1,gp039-gp039/2,gp040-gp040/1,gp040-gp040/2,gp041-gp041/1,gp041-gp041/2,mother-mother/1,mother-mother/2,sum
1030,RRP7A,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0
1094,SMN1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0
244,CYP21A2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
251,DBR1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
297,DPY19L2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
417,FDFT1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0
1415,ZP2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0
107,C4B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1157,SYCE1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
880,PHKG2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [41]:
def get_gene_location_from_gff(gff_file, gene_names):
    """
    Extract the location of a gene from a GFF file.
    
    Args:
    - gff_file (str): Path to the GFF file.
    - gene_name (str): Name of the gene.
    
    Returns:
    - dict: Information about the gene including its location, or None if not found.
    """
    
    found_genes = []
    with open(gff_file, 'r') as file:
        for line in file:
            if line.startswith('#'):  # Skip comment lines
                continue
            fields = line.strip().split('\t')
            if len(fields) < 9:  # A valid GFF row should have 9 columns
                continue
            
            # Check if the current feature is a gene and if the name matches
            feature_type = fields[2]
            attributes = fields[8]
   

            gene_name = '-'.join(attributes.split(';')[0].split('=')[-1].split('-')[1:])
           
            
            if feature_type == "gene" and gene_name in genes:
                found_genes.append( {
                    "Gene": gene_name,
                    "Chromosome": fields[0],
                    "Start": int(fields[3]),
                    "End":int(fields[4]),
                    "Strand": fields[6]
                }
            )
    return found_genes



In [42]:

genes = list(duplication_ratio_matrix['Gene Name'].iloc[:10])
gff_path = "chm13_genomic_chr.gff3"
gene_name = 'RRP7A'
genes_list = get_gene_location_from_gff(gff_path, gene_name)



In [43]:
genes_list

[{'Gene': 'DBR1',
  'Chromosome': 'chr3',
  'Start': 140901580,
  'End': 140915511,
  'Strand': '-'},
 {'Gene': 'SMN1',
  'Chromosome': 'chr5',
  'Start': 71381729,
  'End': 71423141,
  'Strand': '+'},
 {'Gene': 'C4B',
  'Chromosome': 'chr6',
  'Start': 31868001,
  'End': 31888625,
  'Strand': '+'},
 {'Gene': 'CYP21A2',
  'Chromosome': 'chr6',
  'Start': 31891622,
  'End': 31894848,
  'Strand': '+'},
 {'Gene': 'FDFT1',
  'Chromosome': 'chr8',
  'Start': 7900999,
  'End': 7944714,
  'Strand': '-'},
 {'Gene': 'SYCE1',
  'Chromosome': 'chr10',
  'Start': 134505424,
  'End': 134519736,
  'Strand': '-'},
 {'Gene': 'DPY19L2',
  'Chromosome': 'chr12',
  'Start': 63537270,
  'End': 63646941,
  'Strand': '-'},
 {'Gene': 'ZP2',
  'Chromosome': 'chr16',
  'Start': 21128399,
  'End': 21145450,
  'Strand': '-'},
 {'Gene': 'PHKG2',
  'Chromosome': 'chr16',
  'Start': 31135424,
  'End': 31148597,
  'Strand': '+'},
 {'Gene': 'RRP7A',
  'Chromosome': 'chr22',
  'Start': 42989327,
  'End': 43000778,
  '

In [50]:
def parse_gff(gff_file):
    """Extract all genes and their positions from a GFF file."""
    genes = []
    with open(gff_file, 'r') as file:
        for line in file:
            if line.startswith('#'):  # Skip comment lines
                continue
            fields = line.strip().split('\t')
            if len(fields) < 9:  # A valid GFF row should have 9 columns
                continue
            
            # Check if the current feature is a gene
            feature_type = fields[2]
            attributes = fields[8]
            gene_name = '-'.join(attributes.split(';')[0].split('=')[-1].split('-')[1:])
            if feature_type == "gene":
                genes.append({
                    'Gene': fields[8].split(';')[0].split('=')[-1],
                    "Chromosome": fields[0],
                    "Start": int(fields[3]),
                    "End": int(fields[4]),
                    "Strand": fields[6]
                })
    return genes

def find_nearby_genes(target_gene, gff_genes, distance=10000):
    """Find genes within a specified distance from a target gene."""
    nearby = []
    for gene in gff_genes:
        if gene["Chromosome"] == target_gene["Chromosome"]:
            # Check if gene is within the distance upstream or downstream
            if target_gene["Start"] - distance <= gene["End"] <= target_gene["Start"] or target_gene["End"] <= gene["Start"] <= target_gene["End"] + distance:
                nearby.append(gene)
    return nearby

In [51]:
gff_genes = parse_gff(gff_path)

In [52]:
for gene in genes_list:
    nearby_genes = find_nearby_genes(gene, gff_genes)
    
    gene['nearby_genes'] = nearby_genes
    for nearby in nearby_genes:
        print(f"- {nearby['Gene']} {nearby['Chromosome']}:{nearby['Start']}-{nearby['End']} (Strand: {nearby['Strand']})")
    print("\n")



- gene-SERF1A chr5:71357441-71375322 (Strand: +)
- gene-NAIP chr5:71424931-71482091 (Strand: -)


- gene-CYP21A2 chr6:31891622-31894848 (Strand: +)
- gene-TNXB chr6:31894357-31962530 (Strand: -)


- gene-C4B chr6:31868001-31888625 (Strand: +)


- gene-CTSB chr8:7872212-7897789 (Strand: +)
- gene-NEIL2 chr8:7952965-7970597 (Strand: -)




- gene-LOC105369797 chr12:63524154-63530248 (Strand: -)




- gene-SRCAP chr16:31086152-31128418 (Strand: +)
- gene-TMEM265 chr16:31127651-31132204 (Strand: +)
- gene-RNF40 chr16:31149033-31163750 (Strand: +)


- gene-LOC101927372 chr22:42982889-42986839 (Strand: -)




In [54]:
sorted(genes_list, key = lambda x: len(x['nearby_genes']), reverse=True)

[{'Gene': 'PHKG2',
  'Chromosome': 'chr16',
  'Start': 31135424,
  'End': 31148597,
  'Strand': '+',
  'nearby_genes': [{'Gene': 'gene-SRCAP',
    'Chromosome': 'chr16',
    'Start': 31086152,
    'End': 31128418,
    'Strand': '+'},
   {'Gene': 'gene-TMEM265',
    'Chromosome': 'chr16',
    'Start': 31127651,
    'End': 31132204,
    'Strand': '+'},
   {'Gene': 'gene-RNF40',
    'Chromosome': 'chr16',
    'Start': 31149033,
    'End': 31163750,
    'Strand': '+'}]},
 {'Gene': 'SMN1',
  'Chromosome': 'chr5',
  'Start': 71381729,
  'End': 71423141,
  'Strand': '+',
  'nearby_genes': [{'Gene': 'gene-SERF1A',
    'Chromosome': 'chr5',
    'Start': 71357441,
    'End': 71375322,
    'Strand': '+'},
   {'Gene': 'gene-NAIP',
    'Chromosome': 'chr5',
    'Start': 71424931,
    'End': 71482091,
    'Strand': '-'}]},
 {'Gene': 'C4B',
  'Chromosome': 'chr6',
  'Start': 31868001,
  'End': 31888625,
  'Strand': '+',
  'nearby_genes': [{'Gene': 'gene-CYP21A2',
    'Chromosome': 'chr6',
    'Start'

In [22]:
genes_list

[]

In [15]:
telomeric_regions = pd.read_csv('chm13v2.0_telomere.bed', sep='\t', header=None, names=['chromosome', 'start', 'end'])

In [16]:
site_df

,chrom,pos
0,chr1,2096666
1,chr1,2102619
2,chr1,2102916
3,chr1,2160088
4,chr1,5532193
...,...,...
710,chrY,28806400
711,chrY,29335944
712,chrY,29387838
713,chrY,29401036


In [17]:
def in_telomeric_region(row):
    chrom = row['chrom']
    pos = row['pos']
    
    # Filter the telomeric regions for the chromosome
    chrom_regions = telomeric_regions[telomeric_regions['chromosome'] == chrom]
    
    # Check if position falls in any of the intervals
    for _, region in chrom_regions.iterrows():
        if region['start'] <= pos <= region['end']:
            return True
    return False

# Apply the function to each row
site_df['in_telomeric'] = site_df.apply(in_telomeric_region, axis=1)

In [18]:
centromeric_regions = pd.read_csv('CHM13_karyotype.bed', sep='\t')

In [19]:
def in_centromeric_region(row):
    chrom = 'chr' + str(row['chrom']) if isinstance(row['chrom'], int) else row['chrom']
    pos = row['pos']
    
    # Filter the centromeric regions for the chromosome
    chrom_regions = centromeric_regions[centromeric_regions['Chr'] == chrom]
    
    # Check if position falls in any of the intervals
    for _, region in chrom_regions.iterrows():
        if region['CE_start'] <= pos <= region['CE_end']:
            return True
    return False

In [25]:
telomeric_regions

,chromosome,start,end
0,chr1,0,3000
1,chr1,248384000,248387328
2,chr2,0,3600
3,chr2,242693800,242696752
4,chr3,0,2800
5,chr3,200904400,200905400
6,chr3,201101200,201105948
7,chr4,0,3400
8,chr4,193572600,193574945
9,chr5,0,2400


In [20]:
site_df['in_centromeric'] = site_df.apply(in_centromeric_region, axis=1)

In [22]:
site_df[site_df['in_telomeric']]

,chrom,pos,in_telomeric,in_centromeric


In [23]:
site_df

,chrom,pos,in_telomeric,in_centromeric
0,chr1,2096666,False,False
1,chr1,2102619,False,False
2,chr1,2102916,False,False
3,chr1,2160088,False,False
4,chr1,5532193,False,False
...,...,...,...,...
710,chrY,28806400,False,False
711,chrY,29335944,False,False
712,chrY,29387838,False,False
713,chrY,29401036,False,False
